## Description

##### Implementation of "Deep Residual Learning for Image Recognition" paper - https://ieeexplore.ieee.org/document/7780459

##### Implementation of individual model layers

### Libraries

In [ ]:
import tensorflow as tf
import numpy as np
import typing

### Input

##### Let's assume we have an input image of [batch_size, image_width, image_height, features]

In [ ]:
input_shape = (1, 500, 500, 3) # images of batch 100, size of 500x500 and rgb hene feaures of size 3
x = tf.random.normal(input_shape)
x = tf.constant(x, dtype=tf.float32)

In [ ]:
x.shape

### Layers

##### Setup individual layers and test that they work

#### Conv2D

In [ ]:
class Conv2D(tf.Module):
    """
        2D Convolution Layer
    """
    
    def __init__(self,
                out_channels: int, # aka nr. filters
                kernel: int, # the h*w of filter, can be a tuple but for now let's set it to a sqaure
                stride: int, # the h_steps*w_steps of filter as it moves along input, set to a square for now
                padding: str = "SAME",
                use_bias: bool = True,
                name = None
                ):
    
        super(Conv2D, self).__init__(name)
        
        self.out_channels = out_channels
        self.kernel: Tuple[int, int] = (kernel, kernel) # h*w of kernel
        self.stride: Tuple[int, int] = (stride, stride) # h_step*w_step of kernel movement
        self.padding = padding
        self.use_bias = use_bias
        
        self.is_built: bool = False
            
        self.W: Optional[tf.Variable] = None
        self.b: Optional[tf.Variable] = None
            
    
    def __call__(self, x_in: tf.Tensor) -> tf.Tensor:
        """
            Build tensor on the first call.
            Calculate output by 2D convolution.
            
            :param x_in: input tensor of shape (batch_size, in_height, in_width, in_channels)
            
        """
        
        if not self.is_built: # initialize weights
            in_channels = x_in.shape[3]
            filter_weights_shape = self.kernel + (in_channels, self.out_channels)
            
            self.W = tf.Variable(tf.initializers.GlorotUniform()(filter_weights_shape),
                                 trainable = True,
                                 dtype = tf.float32,
                                 name = "conv2d_filters"
                                )
            
            if self.use_bias:
                self.b = tf.Variable(tf.initializers.GlorotUniform()((self.out_channels,)),
                                     trainable = True,
                                     dtype = tf.float32,
                                     name = "conv2d_bias"
                                    )
                
            self.is_built = True # first time weights and bias has been set, this will prevent it from being reset during backprop
        
        conv2d_layer = tf.nn.conv2d(input=x_in, filters=self.W, strides=self.stride, padding=self.padding)
              
        if self.use_bias:
            return tf.add(
                conv2d_layer,
                self.b,
                name = "conv2d_add_bias")
        
        else:
            return conv2d_layer

In [ ]:
conv2d_layer = Conv2D(out_channels=64, kernel=7, stride=2)

In [ ]:
conv2d_layer_output = conv2d_layer(x)
conv2d_layer_output.shape

#### Dense

In [ ]:
class Dense(tf.Module):
    
    def __init__(self, out_features, name=None):
        super().__init__(name=name)
        self.is_built = False # is built flag for dynamic input size inference
        self.out_features = out_features
        
    def __call__(self, x_in):
        if not self.is_built:
            self.w = tf.Variable(
                tf.random.normal([x.shape[-1], self.out_features]), name='w')
            self.b = tf.Variable(tf.zeros([self.out_features]), name='b')
            self.is_built = True
        
        x_hat = tf.matmul(x, self.w) + self.b
        return x_hat

In [ ]:
dense_layer = Dense(out_features=1)

In [ ]:
dense_layer_output = dense_layer(x)
dense_layer_output.shape